In [4]:
# STEP 0 — Import Basic Libraries
import pandas as pd

# STEP 1 - Load data set (heart disease)
from sklearn.datasets import fetch_openml

data = fetch_openml(name="heart-disease", version=1, as_frame=True)
df = data.frame

C:\Users\User\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [6]:
#step 2 - separate feature X and Y
X = df.drop("target", axis=1)
Y = df["target"]

In [7]:
#step 3 - Train/Test split

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [8]:
x_train.shape, x_test.shape


((242, 13), (61, 13))

In [9]:
#step 4 - import randomForest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42
    )

"""n_estimators=100
This tells the model:

“Build 100 decision trees in the forest.”

Why this matters:

More trees → more stable predictions

Fewer trees → faster but less reliable

100 is:

A safe default

Widely used in practice"""

'n_estimators=100\nThis tells the model:\n\n“Build 100 decision trees in the forest.”\n\nWhy this matters:\n\nMore trees → more stable predictions\n\nFewer trees → faster but less reliable\n\n100 is:\n\nA safe default\n\nWidely used in practice'

In [10]:
#step 5 - train randome forest model
rf.fit(x_train, y_train)

"""What Happens in This Step (Simple)
When you run .fit():

The model:

Takes X_train (features)
Takes y_train (correct answers)

Builds 100 decision trees
Each tree:
Sees slightly different data
Learns different rules
The forest stores all learned trees

After this:
The model is trained and ready to make predictions."""

'What Happens in This Step (Simple)\nWhen you run .fit():\n\nThe model:\n\nTakes X_train (features)\nTakes y_train (correct answers)\n\nBuilds 100 decision trees\nEach tree:\nSees slightly different data\nLearns different rules\nThe forest stores all learned trees\n\nAfter this:\nThe model is trained and ready to make predictions.'

In [12]:
# step 6 - make predictions
y_pred_rf = rf.predict(x_test)
y_pred_rf[:10]

"""What’s happening internally (simple)

For each patient in X_test:

Each of the 100 trees makes a prediction

Trees vote

Majority vote becomes the final class (0 or 1)

No probabilities yet — just final decisions."""

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 0], dtype=int64)

In [13]:
# step 7 evaluate randome forest model

#step 7.1 confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_rf)
cm


array([[24,  5],
       [ 5, 27]], dtype=int64)

In [14]:
# step 7.2 accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_rf)

"""What this means

Out of all test patients, the Random Forest predicted about 84% correctly.

Compare with Logistic Regression

Logistic Regression accuracy ≈ 85%

Random Forest accuracy ≈ 84%

👉 So accuracy is slightly lower for Random Forest here.

That’s totally fine — accuracy alone doesn’t decide the better model."""

0.8360655737704918

In [16]:
#step 7.3 recall
from sklearn.metrics import recall_score
recall_score(y_test,y_pred_rf)

"""Medical Insight (Important)

Random Forest did not improve recall here

Missing disease cases is still 5

So from a medical safety point of view:

Random Forest ≈ Logistic Regression

This tells us something important:

Changing the model alone doesn’t always improve recall."""

0.84375

In [17]:
# step 7.4 precision
from sklearn.metrics import precision_score
precision_score(y_test, y_pred_rf)

0.84375

In [ ]:
"""
| Metric              | Logistic Regression | Random Forest |
| ------------------- | ------------------- | ------------- |
| Accuracy            | ~85%                | ~84%          |
| Recall (Disease)    | ~84%                | ~84%          |
| Precision (Disease) | ~87%                | ~84%          |
| Missed disease (FN) | 5                   | 5             |

Random Forest did NOT outperform Logistic Regression here.

Why?

Dataset is small

Relationships may be mostly linear

Default Random Forest settings

No threshold tuning

This is a valuable real-world lesson:

A more complex model is not always better.

-----------------------------------

Medical Perspective (Decision)

If your goal is:

Simple & interpretable → Logistic Regression ✅

Same recall, fewer false alarms → Logistic Regression ✅

So in this case:

Logistic Regression is the better baseline model.
"""

In [21]:
"""FINAL STEP — ROC & AUC (Random Forest)
🧠 Reminder (1 line)

ROC → how performance changes with threshold

AUC → how well the model separates classes overall"""

#We use predict_proba to get how likely each patient has heart disease, and we select the probability of class 1 to calculate ROC and AUC.

#STEP 8.1 — Get Prediction Probabilities

y_prob_rf = rf.predict_proba(x_test)

y_prob_rf_pos = y_prob_rf[:, 1]
y_prob_rf_pos[:5]


array([0.05, 0.62, 0.55, 0.14, 0.79])

In [22]:
#step caculate auc
from sklearn.metrics import roc_auc_score
auc_rf = roc_auc_score(y_test, y_prob_rf_pos)
auc_rf

0.9202586206896551

In [ ]:
"""What this means (simple)

The Random Forest has very good class separation, but slightly worse than Logistic Regression.

Recall:

Logistic Regression AUC ≈ 0.93

Random Forest AUC ≈ 0.92

So:

Both models are strong

Logistic Regression is marginally better here

---------------------------------------------------

On THIS dataset:

Logistic Regression wins

Random Forest does not add value yet

Why this happens:

Dataset is small

Patterns are mostly linear

Random Forest defaults (no tuning)

Same threshold (0.5)

This is a real ML lesson:

More complex ≠ always better

"""